In [1]:
import psycopg2 as pg
import pandas as pd
import os
from datetime import date, datetime, timedelta

In [2]:
def pandafy(rows, colnames):
    N=len(colnames)
    D={cn: [] for cn in colnames}
    for row in rows:
        for i in range(N): D[colnames[i]].append(row[i])
    pdfy=pd.DataFrame(D)
    return pdfy

def excuteSingleSQLstatement(sql, host, database, user, password):
    conn = pg.connect(host=host,database=database, user=user, password=password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    

def getSQLfromQuery(sql, params, host, database, user, password):
    conn = pg.connect(host=host, database=database, user=user, password=password)
    cur  = conn.cursor()    
    if len(params)==0:
        cur.execute(sql)
    else: 
        cur.execute(sql, params)
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    cur.close()
    conn.close()
    df=pandafy(data, colnames)
    return df

#*********************************************
#this is new and good for many insert queries
#*********************************************
def excuteManySQLstatements(sql_list, host, database, user, password):
    conn = pg.connect(host=host,database=database, user=user, password=password)
    cur = conn.cursor()
    for sql in sql_list: cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    

# Conexão

In [3]:
database='Project_db'
password='Tree4seek'

conn = pg.connect(host='localhost', database = database, user = 'postgres', password = password)
conn.close()

# Teste

In [4]:
SQL = """SELECT * FROM tweet_facts limit 2 
           """
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,fk_date,fk_company,volume,diff_openclose,diff_highlow,fk_tweet,comment_num,retweet_num,like_num
0,735600,AMZN,2788101,4,8,550804137118801921,0,0,1
1,735600,AMZN,2788101,4,8,550806376092807168,0,0,1


In [5]:
SQL = """SELECT * FROM dimdate limit 2 
           """
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,pkdate,year,month,day,quarter,semester,weekmonth,weekyear,weekday,daytype
0,735599,2015,1,1,Q1,S1,1,1,Thursday,Working Day
1,735600,2015,1,2,Q1,S1,1,1,Friday,Working Day


In [6]:
SQL = """SELECT * FROM dimcompany limit 2 
           """
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,ticker,name,stock_market,domain,year_founded,industry,size_range,ceo,city,state,country
0,AMZN,amazon,NASDAQ,amazon.com,1994,internet,75000-100000,Jeff Bezos,seattle,washington,united states
1,AAPL,apple,NASDAQ,apple.com,1976,consumer electronics,75000-100000,Tim Cook,cupertino,california,united states


In [7]:
SQL = """SELECT * FROM dimtweet limit 2 
           """
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,tweet_id,writer,body
0,550441509175443456,VisualStockRSRC,"lx21 made $10,008 on $AAPL -Check it out! htt..."
1,550441672312512512,KeralaGuy77,Insanity of today weirdo massive selling. $aap...


# Query 1

## Variação do preço das ações com o número de tweets

In [19]:
SQL = """
SELECT f.fk_date, d.year, d.month, d.day, d.quarter, d.weekday, COUNT(fk_tweet) as "tweet_count", ROUND(AVG(diff_openclose),3) as "Average_Open_Close", ROUND(AVG(diff_highlow),3) as "Average_High_Low"
FROM tweet_facts f, dimdate d
WHERE d.pkdate=f.fk_date
GROUP BY (f.fk_date, d.year, d.month, d.day, d.quarter, d.weekday)
ORDER BY tweet_count desc
limit 15
"""
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,fk_date,year,month,day,quarter,weekday,tweet_count,Average_Open_Close,Average_High_Low
0,736913,2018,8,7,Q3,Tuesday,8213,-28.474,39.608
1,737062,2019,1,3,Q1,Thursday,7430,5.556,10.490
2,735800,2015,7,21,Q3,Tuesday,7056,-0.039,5.720
3,736214,2016,9,7,Q3,Wednesday,6842,-0.184,2.799
4,736264,2016,10,27,Q4,Thursday,6738,5.343,7.999
5,736044,2016,3,21,Q1,Monday,6681,-1.765,5.540
6,735715,2015,4,27,Q2,Monday,6679,0.024,5.344
7,736907,2018,8,1,Q3,Wednesday,6654,-2.778,10.030
8,735625,2015,1,27,Q1,Tuesday,6637,2.901,3.835
9,736128,2016,6,13,Q2,Monday,6608,-0.112,3.002


In [9]:
SQL = """
SELECT f.fk_date, f.fk_company, COUNT(f.fk_tweet) as "tweet_count"
FROM tweet_facts f
GROUP BY (f.fk_date, f.fk_company)
ORDER BY tweet_count desc
"""
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,fk_date,fk_company,tweet_count
0,736913,TSLA,6408
1,736214,AAPL,5038
2,737062,AAPL,4943
3,735666,AAPL,4647
4,736914,TSLA,4624
...,...,...,...
6282,736620,AMZN,12
6283,736401,AAPL,12
6284,736479,AMZN,11
6285,737228,MSFT,9


In [10]:
SQL = """
SELECT c.name, AVG(diff_openclose), COUNT(fk_tweet) as "tweet_count"
FROM tweet_facts f, dimcompany c
WHERE c.ticker=f.fk_company
GROUP BY (c.name)
ORDER BY tweet_count desc
"""
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,name,avg,tweet_count
0,apple,-0.03663239909803947052,1231096
1,tesla,-0.17641949207614525479,923288
2,amazon,0.27429547780087393574,621556
3,google,-0.14061594254124899263,565832
4,microsoft,-0.00955445698473401113,321630


In [12]:
SQL = """
SELECT DISTINCT f.fk_date, c.name, f.diff_openclose
FROM tweet_facts f, dimcompany c
WHERE f.fk_company=c.ticker
ORDER BY f.diff_openclose desc
LIMIT 20
"""
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,fk_date,name,diff_openclose
0,736996,amazon,121
1,736991,amazon,110
2,736977,amazon,103
3,737032,amazon,88
4,737049,amazon,88
5,736733,amazon,79
6,736808,amazon,76
7,737035,amazon,76
8,736780,amazon,75
9,737213,amazon,67


In [13]:
SQL = """
SELECT DISTINCT f.fk_date, c.name, f.diff_openclose
FROM tweet_facts f, dimcompany c
WHERE f.fk_company=c.ticker
ORDER BY f.diff_openclose asc
LIMIT 20
"""
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,fk_date,name,diff_openclose
0,737054,amazon,-102
1,737034,amazon,-84
2,737005,amazon,-82
3,736731,amazon,-81
4,736992,amazon,-79
5,737061,amazon,-74
6,737419,amazon,-68
7,737026,amazon,-64
8,737357,amazon,-64
9,736948,amazon,-59


In [14]:
SQL = """
SELECT c.name AS "Company", ROUND(AVG(f.diff_openclose),3) "Average Growth", SUM(f.diff_openclose) AS "Price Variation"
FROM tweet_facts f, dimcompany c
WHERE f.fk_company=c.ticker
GROUP BY (c.name)
ORDER BY "Price Variation" desc
LIMIT 10
"""
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,Company,Average Growth,Price Variation
0,amazon,0.274,170490
1,microsoft,-0.010,-3073
2,apple,-0.037,-45098
3,google,-0.141,-79565
4,tesla,-0.176,-162886


# Query 2

## Variação do número de tweets ao longo dos anos, por empresa

In [18]:
SQL = """
SELECT d.year, c.name, COUNT(f.fk_tweet) as "tweet_count",
ROUND(AVG(f.diff_openclose),3) as "diff_openclose_average", 
ROUND(AVG(f.diff_highlow),3) as "average_high_low"
FROM tweet_facts f, dimdate d, dimcompany c
WHERE d.pkdate=f.fk_date and c.ticker=f.fk_company
GROUP BY ROLLUP (d.year, c.name)
ORDER BY d.year asc, tweet_count desc
"""
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,year,name,tweet_count,diff_openclose_average,average_high_low
0,2015.0,None,703389,0.051,6.588
1,2015.0,apple,311153,0.191,2.639
2,2015.0,google,151291,-0.312,12.302
3,2015.0,amazon,96338,0.795,12.121
4,2015.0,tesla,95588,-0.488,7.657
5,2015.0,microsoft,49019,-0.132,1.058
6,2016.0,None,817702,0.182,6.392
7,2016.0,apple,355201,-0.076,1.870
8,2016.0,google,151560,0.690,12.741
9,2016.0,amazon,124419,0.105,14.420


# Query 3

## Variação dos tweets por mês

In [15]:
SQL = """
SELECT d.month, COUNT(f.fk_tweet) as "tweet_count"
FROM tweet_facts f, dimdate d
WHERE d.pkdate=f.fk_date 
GROUP BY (d.month)
ORDER BY tweet_count desc
"""
df  = getSQLfromQuery(SQL, (), "localhost", database, "postgres", password)
df

,month,tweet_count
0,10,345450
1,8,343705
2,4,327069
3,5,324351
4,7,314579
5,3,311875
6,9,306393
7,6,297906
8,1,297457
9,2,282130
